In [1]:
library(dplyr)
library(ggplot2)
library(ggpubr)
library(ggrepel)
library(ggpubr)
library(ggh4x)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




# Data load

In [2]:
load('../data/commons.rda', verbose = T)

Loading objects:
  cell.type.colors
  cell.types
  cell.types.nohighmito
  chr_df
  donor_sex
  fig
  flora_paper_list
  gene_type.df
  imprinted.df
  imprinted.genes
  monkey_paper_list
  monkey.genes
  monkey.markers
  nescreg.genes
  nescreg.genes.no_trg
  nescreg.genes.no_trg.early
  nescreg.list
  nescreg.list.no_trg
  nescreg.list.no_trg.early
  nescreg.markers
  nescreg.markers.no_trg
  nescreg.markers.no_trg.early
  nicola_marker_list
  origin_sex.colors
  pat.de
  pat.de.top20
  pat.list
  pat.neu.de
  pat.neu.de.top20
  pat.neu.list
  pat.neu.top20
  pat.top20
  phases
  phases.colors
  protein_coding.genes
  region.genes
  region.list
  region.markers
  samples
  samples.colors
  sex_chr.genes
  sex_chr.genes.x
  sex_chr.genes.y
  shown_2b_list
  shown_4d_list
  tf.genes
  tfs.df
  top100.sub.pat
  top100.subtype
  valid_chr


In [3]:
load('../data/filter.list.rda', verbose=T)

Loading objects:
  exp.filt.opts
  phase.filt.opts
  pval.filt.opts
  tf.filt.opts
  filt.grid
  filt.grid.desc


In [4]:
today_dir <- glue::glue('../plots/vulcanos_{Sys.Date()}/')

dir.create(today_dir)

## Markers

In [5]:
openxlsx::getSheetNames('../results/selected_markers/ASD_vs_Ctrl_markers.Annotated.Filter_None.xlsx')

# markers.all <- openxlsx::read.xlsx('../results/selected_markers/ASD_vs_Ctrl_markers.Annotated.Filter_None.xlsx',
#                                    sheet='Seurat.All')
markers.rge <- openxlsx::read.xlsx('../results/selected_markers/ASD_vs_Ctrl_markers.Annotated.Filter_None.xlsx',
                                   sheet='DESeq2.RGe')
markers.phase <- openxlsx::read.xlsx('../results/selected_markers/ASD_vs_Ctrl_markers.Annotated.Filter_None.xlsx',
                                     sheet='DESeq2.RGe.PerPhase')

[1] "Seurat.All"              "Seurat.All.TFs"         
[3] "DESeq2.RGe"              "DESeq2.RGe.TFs"         
[5] "DESeq2.RGe.PerPhase"     "DESeq2.RGe.PerPhase.TFs"

# Functions

In [6]:
quantile.n <- function(nums, n){
    q <- data.frame(nums=nums, i=1:length(nums), q=0) %>% arrange(nums, na.last=T)
    
    many <- floor(nrow(subset(q, !is.na(nums)))/n)
    for (ni in 1:n){
        init <- many*(ni-1)+1
        fini <- if (ni!=n){many*ni}else{nrow(q)}
        q[init:fini,'q'] <- ni
    }
    q <- q %>% mutate(q=ifelse(is.na(nums), NA, q))
    qs <- (q%>% arrange(i))$q
    return(qs)
}

In [7]:
# Helper to deal with 0 p-values
clip.zero <- function(x, clip=NULL){
    if (is.null(clip)){
        clip <- min(x[x!=0], na.rm = T)
    }
    return(ifelse(x==0, clip, x))
}
# Helper to make any range symmetric
symmetric_limits <- function (x) 
{
    max <- max(abs(x))
    c(-max, max)
}

In [8]:
basic.volcano <- function(df,
                          p_val='p_val', logFC='avg_log2FC', label=NULL,
                          
                          label.groups = NULL, label.colors=NULL, 
                          label.sides=T, label.sides.movement='y',
                          label.segment.curvature = -1e-20, label.segment.alpha=0.4,
                          label.arrow = arrow(length = unit(0.015, "npc")),
                          
                          label.xlim.factor=1, expand.y.factor=0.4, distribute.labels.y = F,
                          
                          unified_non_significants = T,
                          
                          axis.line.p = T, axis.line.FC = T,
                          p.lines = c(.05), fc.lines = c(0.2, 0.5, 1), lines.color='grey',
                          
                          quadrants = c(.05, 0.2), 
                          quadrants.colors = c('tomato', 'blue', 'grey', 'grey', 'lightgrey'), 
                          
                          alpha.quadrants = T, 
                          quadrants.alphas = c(0.9, .5,.5,.5),
                          
                          fill.logFC = T, palette = 'turbo', palette.type='viridis',
                          symmetric_logFC=T,
                          
                          xlab='log2FC', ylab='-log10(p)',
                          title='Vulcano plot', subtitle=NULL,
                          
                          facet.wrap = NULL,
                          
                          size.point = 2, size.stroke=0.1, size.text=3,
                          
                          theming=T, ...
                         ){
    
    #################
    # PREPARE
    #################
    require(dplyr)
    require(ggplot2)
    require(ggpubr)
    require(ggrepel)
    require(ggpubr)
    require(ggh4x)    

    dfp <- df 
    
    # Standard columns (axes)
    #################
    dfp['p_val'] <- dfp[,p_val]
    dfp['logFC'] <- dfp[,logFC]
    if (!is.null(facet.wrap)){
        dfp['facet'] <- dfp[,facet.wrap]
    }
    
    # Define quadrants
    #################
    q.lvls <- c(glue::glue('p<={quadrants[1]} & |logFC|>={quadrants[2]}'),
                glue::glue('p<={quadrants[1]} & |logFC|<{quadrants[2]}'),
                glue::glue('p>{quadrants[1]} & |logFC|>={quadrants[2]}'),
                glue::glue('p>{quadrants[1]} & |logFC|<{quadrants[2]}'))
                             
    if (unified_non_significants){
        q.lvls[2:4] <- 'NS'
    }
    
    # Get quadrant
    dfp['quadrant'] <- paste(ifelse(dfp[,'p_val'] <= quadrants[1], 
                                    glue::glue('p<={quadrants[1]}'), glue::glue('p>{quadrants[1]}')),
                             ifelse(abs(dfp[,'logFC']) >= quadrants[2], 
                                    glue::glue('|logFC|>={quadrants[2]}'), glue::glue('|logFC|<{quadrants[2]}')),
                             sep=' & ')
    # Get if it is in significant quadrant
    dfp['q.sig'] <- dfp$quadrant==q.lvls[1]
    fill.logFC <- fill.logFC & any(dfp$q.sig)
                             
    if (length(quadrants.colors) == 5){
        dfp <- dfp %>% mutate(quadrant = ifelse(quadrant == q.lvls[1],
                                                paste(ifelse(logFC>0, 'UP', 'DOWN'), quadrant),
                                                quadrant))
        q.lvls <- c(paste('UP', q.lvls[1]), paste('DOWN', q.lvls[1]), q.lvls[-1])
        if(length(quadrants.alphas)==4){
            quadrants.alphas <- c(quadrants.alphas[1], quadrants.alphas)
        }
    }
    if (unified_non_significants){
        quadrants.colors <- rev(rev(quadrants.colors)[-c(2:3)])
        q.lvls <- rev(rev(q.lvls)[-c(2:3)])
    } 
    q.colors <- setNames(quadrants.colors, q.lvls)
    dfp['quadrant'] <- factor(ifelse(dfp[,'quadrant']%in%q.lvls, dfp[,'quadrant'], 'NS'), levels=unique(q.lvls))
                              

                             
    
    
    # Get labels
    #################
    if (!is.null(label)){
        dfp['label'] <- ifelse(dfp[,'q.sig'], dfp[,label], NA)
        dfp['label.color'] <- if (!is.null(label.groups)){dfp[,label.groups]} else {NA}
    }
    
    # return(dfp)
    
    #################
    ## PLOT
    #################
    x.lim <- max(abs(range(dfp['logFC'])), na.rm=T)
    y.lim <- max(-log10(clip.zero(dfp$p_val)))
    y.lim.nolog <- min(clip.zero(dfp$p_val))
    
    # Basic axes
    p <- ggplot(dfp, aes(x=logFC, y=-log10(clip.zero(p_val))))

    # Symmetric logFC axis
    if (symmetric_logFC){ p <- p + scale_x_continuous(limits = symmetric_limits)}
    
    # Labels and titles
    p <- p + labs(x=xlab, y=ylab)
    p <- p + ggtitle(title, subtitle)
    
    
    # Geom_point
    #################
    gpoint_aes <- aes()
    ## Fill
    gpoint_aes$fill <- if (fill.logFC){quote(ifelse(q.sig, logFC, NA)) } else {quote(quadrant) }
    ## Alpha
    if (alpha.quadrants){
        gpoint_aes$alpha <- quote(quadrant)
    }
    
    p <- p + geom_point(mapping=gpoint_aes,
                        shape=21, stroke=size.stroke, size=size.point)
    
    # Scales
    #################
    ## Fill
    if (fill.logFC){
        if (palette.type=='viridis'){
            p <- p + viridis::scale_fill_viridis(xlab, option = palette, limits = symmetric_limits, na.value=rev(q.colors)[1])
        } else if(palette.type=='distiller'){
            p <- p + scale_fill_distiller(xlab, palette = palette, limits = symmetric_limits, na.value=rev(q.colors)[1])
        } else if(palette.type == 'colorspace'){
            p <- p + scale_fill_continuous_diverging(palette, name=xlab, limits = symmetric_limits, na.value=rev(q.colors)[1])

        }
    } else {
        p <- p + scale_fill_manual('Significance', values=q.colors)
    }
    ## Alpha
    if (alpha.quadrants){
        p <- p + scale_alpha_manual('Significance', values=quadrants.alphas)
    }

    # Lines
    #################
    ## p-val
    if (axis.line.p){p <- p + geom_hline(yintercept=0)}
    if (!is.null(p.lines)){ for (l in p.lines){ p <- p + geom_hline(yintercept=-log10(l), color=lines.color, linetype = "dashed") }}
    ## logFC
    if (axis.line.FC){p <- p + geom_vline(xintercept=0)}
    if (!is.null(fc.lines)){ for (l in fc.lines){ p <- p + geom_vline(xintercept=c(-l,l), color=lines.color, linetype = "dashed") }}
    
    
    
    # Labels of genes
    #################
    if (!is.null(label) & any(!is.na(dfp$label))){
        
        # p <- p +geom_text(aes(label=label))
        repel.aes <- aes(label=label, nudge_x=nudge_x, color=label.color, nudge_y =label.y)
        if (is.null(label.colors)) { repel.aes <- repel.aes[-3]}
        
        
        if (!label.sides){ 
            repel.aes <- repel.aes[-2]
        } else {
            
            p$data$nudge_x <- if (length(label.xlim.factor)>1){
                cut(p$data$p_val,
                    include.lowest = T, 
                    quantile(subset(p$data, !is.na(label))$p_val, probs = c(0, 1/(rev(1:length(label.xlim.factor))))), 
                    labels=c(label.xlim.factor)) %>% as.character%>%as.numeric 
            } else {label.xlim.factor}
            
            p$data$nudge_x <- sign(p$data[,'logFC'])*x.lim*p$data$nudge_x
            
            if (max(label.xlim.factor)> 1){
                if (symmetric_logFC){ 
                    p <- p + scale_x_continuous(
                        expand = expansion(mult=c(max(label.xlim.factor)-1, max(label.xlim.factor)-1)),
                        limits = symmetric_limits)
                } else { 
                    p <- p +scale_x_continuous(expand = expansion(mult=c(NA, max(label.xlim.factor)-1)))
                }
            }
        }
        

        
        if (distribute.labels.y){
            
            if (length(label.xlim.factor)==1){
                adapted.y <- (subset(p$data, !is.na(label)) %>% group_by(sign(logFC)) %>% 
                    mutate(adapted.y = scales::rescale(rank(padj, ties.method = 'random'),
                                                       to = c(0, y.lim)))) %>% as.data.frame()


                p$data$label.y <- ifelse(!is.na(p$data$label),
                                         plyr::mapvalues(p$data$label, from=adapted.y$label, to=adapted.y$adapted.y),
                                         NA) %>% as.numeric()
            } else {
                n.div <- length(label.xlim.factor)
                
                
                p$data <- p$data %>% group_by(sign(logFC)) %>% mutate(qs=quantile.n(ifelse(is.na(label), NA, p_val), n.div),
                                                                      qs=ifelse(is.na(qs), 0, qs)) %>% as.data.frame()
                p$data <- lapply(split(p$data, p$data$qs),
                                 function(pdata){
                                     qs <- unique(pdata$qs)
                                     if(all(qs==0)){
                                         pdata$label.y <- NA
                                     } else {
                                         pdata$label.y <- scales::rescale(
                                             rank(pdata$padj, ties.method = 'random'),
                                             to = c(y.lim/n.div*(qs-1), y.lim/n.div*qs))
                                     }
                                     return(pdata)
                                 }) %>% do.call(what='rbind')  %>% as.data.frame()
            }
        } else{
            repel.aes <- repel.aes[-length(repel.aes)]
        }
        
        p <- p + geom_text_repel(
            mapping=repel.aes, 
            size=size.text, 
            segment.alpha=label.segment.alpha,
            direction = label.sides.movement, 
            segment.curvature = label.segment.curvature,
            arrow=label.arrow,
            ...)
        
        if (!is.null(label.colors) ){
            p <- p + scale_color_manual(label.groups, values=label.colors)
        }
    }
    
    # Facets
    #################
    if (!is.null(facet.wrap)){
        p <- p + facet_wrap2(facets = vars(facet), scales = 'free')
    }
    # Make 0 the bottom
    p <- p + scale_y_continuous(expand = expansion(mult=c(0, expand.y.factor)))
    # Theming
    #################
    if (theming){p <- p + theme_minimal() + labs_pubr()}
    
    return(p)
}

In [9]:
safe.function <- function(fun, safe.ggplot=F, ...){
    par.safe <- list(...)
    
    safe.wrap <- function(...){
        x <- tryCatch({
            x <- rlang::exec(fun, ...)
            if (safe.ggplot){
                y <- ggplot2::ggplot_build(x)
                eval(y)
                }
            return(x)
        }, cond= function(cond){
            message(glue::glue('Condition in unsafe function. Running in safe mode. {as.character(cond)}'))
            rlang::exec(fun, !!!par.safe, ...)
        }, error= function(error){
            message(glue::glue('ERROR in unsafe function. Running in safe mode. {as.character(error)}'))
            rlang::exec(fun, !!!par.safe, ...)
        })
        
        return(x)
    }
    
    return(safe.wrap)
}

In [10]:
safe.volcano <- safe.function(basic.volcano, plot.sides = F, safe.ggplot=T)

In [11]:
compute_rank <- function(x, mode='pval', mask=NULL){
    
    r <- rep(Inf, length.out = length(x))
    
    if (is.null(mask)){
        mask <- rep(T, length.out = length(x))
    }
    
    if (mode %in% c('pval', 'p', 'p-val', 'p_val', 'pvalue')){
        r[mask] <- rank(x[mask], ties.method = 'random')
    } else {
        r[mask] <- rank(-abs(x[mask]), ties.method = 'random')
    }
    return(r)
}

# Plots

## Pseudobulk

In [12]:
# Colors
col.labs <- setNames(
    c('TRUE TRUE', 'TRUE FALSE', 'FALSE TRUE', 'FALSE FALSE'),
    c('TF in sex chr.', 'TF', 'Gene in sex chr.', 'Gene'))
volc.colors <- setNames(
    c('#ff7878', 'darkred', 'grey', 'black'), names(col.labs))

top.n.gral <- 30
n_by_side <- T
padj.thres <- .05
logfc.thres <- 0.5 #round(log2(1.5), 2)

max.overlaps <- Inf
max.iter <- 2E6
max.time <- 10
label.segment.curvature <- 0


In [13]:
check_ranks <- c('FOXG1')
check_ranks_only <- F

In [14]:
fig(10,10)

In [15]:
-log(1/10^12.5)

[1] 28.78231

In [16]:
# this is filthy
p.val.max <- 1e-100
p.val.override <- 1/(10^12.5)
p.val.override

[1] 3.162278e-13

In [17]:
for (gene.lab in c('label_top_lfc')){
    
    # cairo_pdf(glue::glue('{today_dir}pseudobulk_rge.{gene.lab}_notENSID.pdf'), height=8, width=10, onefile = T)
    # for(exp.filt in c('filter.None', names(exp.filt.opts))){
    for(exp.filt in c('filter.None'
                      #'filter.2of3.over.2of3'
                      #'filter.2of3.over.max','filter.all.over.2of3','filter.all.over.max'
                     )){

#         match.files <- list.files(today_dir) %>% grep(pattern=gene.lab, value=T) %>% grep(pattern='per_phase', invert=T, value=T)
#         unique(sapply(strsplit(match.files, split='.filter.', fixed=T), 
#                       function(x){gsub(x[2], pattern='.pdf', fixed=T, replacement='')})) %>% paste0('filter.',.)-> done.filts 

#         filts.to.do <- c('filter.None', names(exp.filt.opts))[! c('filter.None', names(exp.filt.opts)) %in% done.filts]
#         if (length(filts.to.do)==0){
#             break
#         }

#         exp.filt <- filts.to.do[1]
        print(exp.filt)
        top.n <- top.n.gral

        pseudobulk.res <- subset(markers.rge, !gene%in%sex_chr.genes)

        # fill na and remove 0
        pseudobulk.res <- subset(pseudobulk.res, baseMean!=0)
        pseudobulk.res <- pseudobulk.res %>% mutate(padj = ifelse(is.na(padj),1, padj))

        # categorise genes
        pseudobulk.res$`Gene class` <- plyr::mapvalues(
            x = paste(pseudobulk.res$gene%in%tf.genes, pseudobulk.res$gene%in%sex_chr.genes),
            from = col.labs, to=names(col.labs))

        # Apply filter
        pseudobulk.res$filt <- if (exp.filt == 'filter.None'){T} else{ pseudobulk.res[,exp.filt]}
        pseudobulk.res$filt <- pseudobulk.res$filt & 
            (pseudobulk.res$padj < padj.thres)&
            (abs(pseudobulk.res$log2FoldChange)>=logfc.thres) &
            # pseudobulk.res$protein_coding
            !stringr::str_starts(string = pseudobulk.res$gene, pattern = 'ENSG')
        
        
        to_rank <- if (!n_by_side){ list(pseudobulk.res)
                                  } else {to_rank <- split(pseudobulk.res, pseudobulk.res$overexpressed.in)}
        
        lapply(to_rank, function(res){
            
            res$rank.lfc <- compute_rank(res$log2FoldChange, mask = res$filt, mode='lfc')
            res$rank.pval <- compute_rank(res$padj, mask = res$filt, mode='pval')
            
            res$do.label.rank <- if (gene.lab == 'label_top_both'){
                rank(apply(res[,c('rank.lfc', 'rank.pval')], 1, mean))
            }else if(gene.lab == 'label_top_lfc'){res$rank.lfc}else{res$rank.pval}
                
            return(res)
        }) %>% do.call(what='rbind') %>% as.data.frame() -> pseudobulk.res
        
        
        if (!is.null(check_ranks)){
            
            checked_ranks <- subset(pseudobulk.res, gene%in%check_ranks)[,c('gene', 'do.label.rank')]
            min.rank <- max(checked_ranks[is.finite(checked_ranks[,2]),2])
            
            if (!is.null(min.rank)){ if(min.rank >top.n){
                z <- if (n_by_side){5}else{10}
                top.n <- ceiling(min.rank/z)*z
                print(glue::glue('Updading number of genes {top.n.gral} < {min.rank} ->>> {top.n}'))
            }}
                        
            if (check_ranks_only){ next() }
        }
        
        pseudobulk.res$do.label <- pseudobulk.res$do.label.rank <= top.n
        # delete this 
        pseudobulk.res$do.label <- pseudobulk.res$do.label | (pseudobulk.res$padj < p.val.override)
        pseudobulk.res[, gene.lab] <- ifelse(pseudobulk.res$do.label, pseudobulk.res$gene, NA)

        basic.volcano(
            pseudobulk.res, 

            p.lines = c(padj.thres),
            fc.lines = c(logfc.thres),

            size.text = if(p.val.override > -Inf){1.5}else{3},
            distribute.labels.y = F,
            quadrants.colors =  c('tomato', 'blue', 'grey', 'grey', 'darkgrey'),
            fill.logFC = F,
            expand.y.factor = 0.01,
            label.xlim.factor = rev(c(1.3, 1.5, 1.5)),
            # label.xlim.factor =1.5,

            quadrants = c(padj.thres, logfc.thres),
            # palette = 'Blue-Red 2', palette.type='colorspace', 
            palette='turbo', palette.type='viridis',
            label.sides.movement='y',

            title='ASD vs Control gene expression in Early Radial Glia', 
            subtitle=glue::glue('Labeled top {sum(!is.na(pseudobulk.res[,gene.lab]))} genes over- and underexpressed (adj. p-val < {padj.thres})\n{if (exp.filt=="filter.None"){""}else{exp.filt.opts[[exp.filt]]}}'),

            p_val = 'padj', logFC = 'log2FoldChange',
            label = gene.lab, label.sides = T,


            label.groups='Gene class', label.colors=volc.colors, label.segment.alpha=0.3,
            max.overlaps=max.overlaps, max.iter=max.iter, max.time=max.time, label.segment.curvature=label.segment.curvature) -> p3


        p3$data$p_val[p3$data$p_val < p.val.max] <- p.val.max
        p3 + scale_y_continuous(labels=function(x){c(head(x,-1), paste('>', tail(x,1)))}) -> p3
        
        # plot(p3)
        # data.table::fwrite(p3$data, glue::glue('{today_dir}pseudobulk_rge.{gene.lab}_notENSID.table.{exp.filt}.csv'))
    }
    # dev.off()    
}
# plot(p3)


[1] "filter.None"


The following `from` values were not present in `x`: TRUE TRUE, FALSE TRUE

Scale for x is already present.
Adding another scale for x, which will replace the existing scale.
Warning message in geom_text_repel(mapping = repel.aes, size = size.text, segment.alpha = label.segment.alpha, :
“Ignoring unknown aesthetics: nudge_x”
Scale for y is already present.
Adding another scale for y, which will replace the existing scale.


### Supplementary table 8

In [29]:
dfs <- subset(p3$data, !in.sex_chr)[,c(
    'gene', 'baseMean', 'log2FoldChange', 'lfcSE', 'stat', 'pvalue', 'padj',
    'gene.is.tf', 'gene_biotype', 'Imprinted.Status','Expressed.Allele')] %>% 
    mutate(Up.in.ASD.or.control = plyr::mapvalues(sign(log2FoldChange), from=c(-1,0,1), to=c('Control', 'None', 'ASD'))) %>%
    arrange(Up.in.ASD.or.control, desc(abs(log2FoldChange)))

dfs <- list(
    'Filtered pseudobulk DEGs' = subset(dfs) %>% subset((padj <= .05) & (abs(log2FoldChange) >= 0.5)),
    'Pseudobulk DEGs' = dfs)

openxlsx::write.xlsx(x = dfs, file = '../results/supp_table.8.pseudobulk_DEGs.xlsx')

In [19]:
subset(pseudobulk.res, gene =='FOXG1')

,gene,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,gene.is.tf,Imprinted.Status,Expressed.Allele,⋯,phase.bias,same.bias.with.all.phases,same.bias.with.any.phase,Gene class,filt,rank.lfc,rank.pval,do.label.rank,do.label,label_top_lfc
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<chr>,<chr>,⋯,<chr>,<lgl>,<lgl>,<chr>,<lgl>,<dbl>,<dbl>,<dbl>,<lgl>,<chr>
ASD.19281,FOXG1,18.446,4.515627,0.4114345,10.97532,5.022538e-28,8.188615e-26,TRUE,Predicted,Paternal,⋯,UP,TRUE,TRUE,TF,TRUE,5,48,5,TRUE,FOXG1


In [27]:
pseudobulk.res <- pseudobulk.res %>% mutate(is.de = (padj < padj.thres)&(abs(log2FoldChange) > logfc.thres),
                                            is.up = is.de & (log2FoldChange>0),
                                            is.down = is.de & (log2FoldChange<0),
                                            # is.imprinted = Imprinted.Status %in%c('Predicted', 'Imprinted'))
                                            is.imprinted = Imprinted.Status %in%c('Imprinted'),
                                            is.maternal = is.imprinted & (Expressed.Allele == 'Maternal'),
                                            is.paternal = is.imprinted & (Expressed.Allele == 'Paternal')
                                           )

In [28]:
duplicated(pseudobulk.res$gene) %>% sum
unique(pseudobulk.res$Expressed.Allele)

[1] 0

[1] "Not in DB"         "Maternal"          "Paternal"         
[4] "Biallelic"         "Isoform Dependent" "Random"           
[7] "Unknown"           ""

In [53]:
lnc.enrich <- table(subset(pseudobulk.res, !in.sex_chr)%>%mutate(is.lnc = gene_biotype == 'lncRNA')%>%select(is.de, is.lnc))[2:1,2:1]
lnc.enrich.test <- fisher.test(lnc.enrich, alternative = 'greater')
lnc.enrich
lnc.enrich.test


lnc.enrich.up <- table(subset(pseudobulk.res, !in.sex_chr)%>%mutate(is.lnc = gene_biotype == 'lncRNA')%>%select(is.up, is.lnc))[2:1,2:1]
lnc.enrich.up.test <- fisher.test(lnc.enrich.up, alternative = 'greater')
lnc.enrich.up
lnc.enrich.up.test



lnc.enrich.down <- table(subset(pseudobulk.res, !in.sex_chr)%>%mutate(is.lnc = gene_biotype == 'lncRNA')%>%select(is.down, is.lnc))[2:1,2:1]
lnc.enrich.down.test <- fisher.test(lnc.enrich.down, alternative = 'greater')
lnc.enrich.down
lnc.enrich.down.test

       is.lnc
is.de    TRUE FALSE
  TRUE    294   965
  FALSE  9781 15368


	Fisher's Exact Test for Count Data

data:  lnc.enrich
p-value = 1
alternative hypothesis: true odds ratio is greater than 1
95 percent confidence interval:
 0.4268724       Inf
sample estimates:
odds ratio 
 0.4787043 


       is.lnc
is.up    TRUE FALSE
  TRUE    187   721
  FALSE  9888 15612


	Fisher's Exact Test for Count Data

data:  lnc.enrich.up
p-value = 1
alternative hypothesis: true odds ratio is greater than 1
95 percent confidence interval:
 0.3555549       Inf
sample estimates:
odds ratio 
  0.409517 


       is.lnc
is.down  TRUE FALSE
  TRUE    107   244
  FALSE  9968 16089


	Fisher's Exact Test for Count Data

data:  lnc.enrich.down
p-value = 0.9989
alternative hypothesis: true odds ratio is greater than 1
95 percent confidence interval:
 0.5793052       Inf
sample estimates:
odds ratio 
 0.7078162 


In [43]:
impr.enrich <- table(subset(pseudobulk.res, !in.sex_chr)%>%select(is.de, is.imprinted))[2:1,2:1]
impr.enrich.test <- fisher.test(impr.enrich, alternative = 'greater')
impr.enrich
impr.enrich.test

       is.imprinted
is.de    TRUE FALSE
  TRUE     19  1240
  FALSE    80 25069


	Fisher's Exact Test for Count Data

data:  impr.enrich
p-value = 1.909e-07
alternative hypothesis: true odds ratio is greater than 1
95 percent confidence interval:
 2.997872      Inf
sample estimates:
odds ratio 
  4.801473 


In [54]:
impr.enrich.up <- table(subset(pseudobulk.res, !in.sex_chr)%>%select(is.up, is.imprinted))[2:1,2:1]
impr.enrich.up.test <- fisher.test(impr.enrich.up, alternative = 'greater')
impr.enrich.up
impr.enrich.up.test

       is.imprinted
is.up    TRUE FALSE
  TRUE     12   896
  FALSE    87 25413


	Fisher's Exact Test for Count Data

data:  impr.enrich.up
p-value = 0.0001507
alternative hypothesis: true odds ratio is greater than 1
95 percent confidence interval:
 2.175164      Inf
sample estimates:
odds ratio 
  3.911884 


In [55]:
impr.enrich.down <- table(subset(pseudobulk.res, !in.sex_chr)%>%select(is.down, is.imprinted))[2:1,2:1]
impr.enrich.down.test <- fisher.test(impr.enrich.down, alternative = 'greater')
impr.enrich.down
impr.enrich.down.test

       is.imprinted
is.down  TRUE FALSE
  TRUE      7   344
  FALSE    92 25965


	Fisher's Exact Test for Count Data

data:  impr.enrich.down
p-value = 0.0003608
alternative hypothesis: true odds ratio is greater than 1
95 percent confidence interval:
 2.610641      Inf
sample estimates:
odds ratio 
  5.742012 


In [44]:
mat.impr.enrich <- table(subset(pseudobulk.res, !in.sex_chr)%>%select(is.de, is.maternal))[2:1,2:1]
mat.impr.enrich.test <- fisher.test(mat.impr.enrich, alternative = 'greater')
mat.impr.enrich
mat.impr.enrich.test

       is.maternal
is.de    TRUE FALSE
  TRUE      8  1251
  FALSE    24 25125


	Fisher's Exact Test for Count Data

data:  mat.impr.enrich
p-value = 9.849e-05
alternative hypothesis: true odds ratio is greater than 1
95 percent confidence interval:
 3.019694      Inf
sample estimates:
odds ratio 
  6.692275 


In [45]:
up.mat.impr.enrich <- table(subset(pseudobulk.res, !in.sex_chr)%>%select(is.up, is.maternal))[2:1,2:1]
up.mat.impr.enrich.test <- fisher.test(up.mat.impr.enrich, alternative = 'greater')
subset(pseudobulk.res, !in.sex_chr&is.up&is.maternal)%>%select(gene, is.up, is.maternal, everything())
up.mat.impr.enrich
up.mat.impr.enrich.test

,gene,is.up,is.maternal,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,gene.is.tf,⋯,filt,rank.lfc,rank.pval,do.label.rank,do.label,label_top_lfc,is.de,is.down,is.imprinted,is.paternal
,<chr>,<lgl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,⋯,<lgl>,<dbl>,<dbl>,<dbl>,<lgl>,<chr>,<lgl>,<lgl>,<lgl>,<lgl>
ASD.4894,DIO3OS,TRUE,TRUE,2.454944,1.6332275,0.3754242,4.350352,1.359189e-05,1.368553e-04,FALSE,⋯,TRUE,102,406,102,FALSE,NA,TRUE,FALSE,TRUE,FALSE
ASD.20398,H19,TRUE,TRUE,4.476916,1.2347929,0.2623510,4.706644,2.518278e-06,3.102498e-05,FALSE,⋯,TRUE,219,354,219,FALSE,NA,TRUE,FALSE,TRUE,FALSE
ASD.26145,MEG3,TRUE,TRUE,142.954244,3.7573414,0.1183913,31.736628,4.856293e-221,8.471804e-217,FALSE,⋯,TRUE,9,1,9,TRUE,MEG3,TRUE,FALSE,TRUE,FALSE
ASD.26146,MEG8,TRUE,TRUE,29.041581,3.1389680,0.1742565,18.013492,1.526866e-72,1.902584e-69,FALSE,⋯,TRUE,16,8,16,TRUE,MEG8,TRUE,FALSE,TRUE,FALSE
ASD.27863,NTM,TRUE,TRUE,8.146661,0.6734064,0.2078325,3.240140,1.194710e-03,6.688612e-03,FALSE,⋯,TRUE,572,604,572,FALSE,NA,TRUE,FALSE,TRUE,FALSE
ASD.28527,OSBPL5,TRUE,TRUE,7.072213,0.5198947,0.1663144,3.125976,1.772163e-03,9.317473e-03,FALSE,⋯,TRUE,781,637,781,FALSE,NA,TRUE,FALSE,TRUE,FALSE


       is.maternal
is.up    TRUE FALSE
  TRUE      6   902
  FALSE    26 25474


	Fisher's Exact Test for Count Data

data:  up.mat.impr.enrich
p-value = 0.0006846
alternative hypothesis: true odds ratio is greater than 1
95 percent confidence interval:
 2.615557      Inf
sample estimates:
odds ratio 
  6.516151 


In [46]:
pat.impr.enrich <- table(subset(pseudobulk.res, !in.sex_chr)%>%select(is.de, is.paternal))[2:1,2:1]
pat.impr.enrich.test <- fisher.test(pat.impr.enrich, alternative = 'greater')
pat.impr.enrich
pat.impr.enrich.test

       is.paternal
is.de    TRUE FALSE
  TRUE      8  1251
  FALSE    47 25102


	Fisher's Exact Test for Count Data

data:  pat.impr.enrich
p-value = 0.004327
alternative hypothesis: true odds ratio is greater than 1
95 percent confidence interval:
 1.611558      Inf
sample estimates:
odds ratio 
   3.41511 


In [47]:
tf.impr.enrich <- table(subset(pseudobulk.res, (!in.sex_chr)&gene.is.tf)%>%select(is.de, is.imprinted))[2:1,2:1]
tf.impr.enrich.test <- fisher.test(tf.impr.enrich, alternative = 'greater')
tf.impr.enrich
tf.impr.enrich.test

       is.imprinted
is.de   TRUE FALSE
  TRUE     3   128
  FALSE   18  1310


	Fisher's Exact Test for Count Data

data:  tf.impr.enrich
p-value = 0.2903
alternative hypothesis: true odds ratio is greater than 1
95 percent confidence interval:
 0.4221044       Inf
sample estimates:
odds ratio 
  1.704953 


In [70]:
unique(subset(pseudobulk.res, !(in.sex_chr) & (padj < padj.thres)&(abs(log2FoldChange) > logfc.thres))%>% 
           select(log2FoldChange)) %>% 
    mutate(log2FoldChange=sign(log2FoldChange))%>%
    table() -> a
unique(subset(pseudobulk.res, !(in.sex_chr) & (padj < padj.thres)&(abs(log2FoldChange) > logfc.thres) & gene.is.tf)%>% 
           select(log2FoldChange)) %>% 
    mutate(log2FoldChange=sign(log2FoldChange))%>%
    table() -> b
# unique(subset(pseudobulk.res, !(in.sex_chr) & (padj < padj.thres)&(abs(log2FoldChange) > logfc.thres)&gene.is.tf)$gene)%>%length()
print(padj.thres)
print(logfc.thres)
a
sum(a)
b
sum(b)

[1] 0.05
[1] 0.5


log2FoldChange
 -1   1 
351 908 

[1] 1259

log2FoldChange
-1  1 
40 91 

[1] 131

### PSEUDOBULK PER PHASE

In [18]:
for (gene.lab in c('label_top_lfc', 'label_top_pva')){
    # top.n <- if (gene.lab=='label_top_lfc'){50}else{70}
    cairo_pdf(glue::glue('{today_dir}pseudobulk_rge.per_phases.{gene.lab}_notENSID.pdf'), height=8, width=20, onefile = T)
    # for(exp.filt in c('filter.None', names(exp.filt.opts))){
    for(exp.filt in c('filter.None')){

        print(exp.filt)

        # data
        marker_dfs <- lapply(split(markers.phase, markers.phase$Phase), function(x){

            top.n <- top.n.gral

            # fill na and remove 0
            pseudobulk.res <- subset(x, (baseMean!=0) & !gene%in%sex_chr.genes)
            pseudobulk.res <- pseudobulk.res %>% mutate(padj = ifelse(is.na(padj),1, padj))

            # categorise genes
            pseudobulk.res$`Gene class` <- plyr::mapvalues(
                x = paste(pseudobulk.res$gene%in%tf.genes, pseudobulk.res$gene%in%sex_chr.genes),
                from = col.labs, to=names(col.labs))

            # Apply filter
            pseudobulk.res$filt <- if (exp.filt == 'filter.None'){T} else{ pseudobulk.res[,exp.filt]}
            pseudobulk.res$filt <- pseudobulk.res$filt & 
                (pseudobulk.res$padj < padj.thres)&
                (abs(pseudobulk.res$log2FoldChange)>=logfc.thres)#&
                # pseudobulk.res$protein_coding
                # !stringr::str_starts(string = pseudobulk.res$gene, pattern = 'ENSG')


            to_rank <- if (!n_by_side){ list(pseudobulk.res)
                                      } else {to_rank <- split(pseudobulk.res, pseudobulk.res$overexpressed.in)}

            lapply(to_rank, function(res){
                res$do.label.rank <- if(gene.lab == 'label_top_lfc'){compute_rank(res$log2FoldChange, mask = res$filt, mode='lfc')        
                                                                    }else{compute_rank(res$padj, mask = res$filt, mode='pval')}
                return(res)
            }) %>% do.call(what='rbind') %>% as.data.frame() -> pseudobulk.res


            if (!is.null(check_ranks)){

                checked_ranks <- subset(pseudobulk.res, gene%in%check_ranks)[,c('gene', 'do.label.rank')]
                min.rank <- max(checked_ranks[is.finite(checked_ranks[,2]),2])

                if (!is.null(min.rank)){ if(min.rank >top.n){
                    z <- if (n_by_side){5}else{10}
                    top.n <- ceiling(min.rank/z)*z
                    print(glue::glue('Updading number of genes {top.n.gral} < {min.rank} ->>> {top.n}'))
                }}

                if (check_ranks_only){ next() }
            }
            pseudobulk.res$do.label <- pseudobulk.res$do.label.rank <= top.n
            pseudobulk.res[, gene.lab] <- ifelse(pseudobulk.res$do.label, pseudobulk.res$gene, NA)

            
            pseudobulk.res
        })

        # plots
        lapply(marker_dfs[c(1,3,2)], function(x){
               basic.volcano(x,
                distribute.labels.y = F,
                quadrants.colors =  c('tomato', 'blue', 'grey', 'grey', 'darkgrey'),
                fill.logFC = F,
                p.lines = c(padj.thres),
                fc.lines = c(logfc.thres),
                expand.y.factor = 0.01,
                label.xlim.factor = rev(c(1.3, 1.5, 1.5)),
                # label.xlim.factor =1.5,

                quadrants = c(padj.thres, logfc.thres),
                # palette = 'Blue-Red 2', palette.type='colorspace', 
                palette='turbo', palette.type='viridis',
                label.sides.movement='y',

                title=glue::glue('ASD vs Control gene expression in {unique(x$Phase)} (Early Radial Glia)'), 
                subtitle=glue::glue(
                    'Labeled top {sum(!is.na(x[,gene.lab]))} genes over- and underexpressed (adj. p-val < {padj.thres})\n{if (exp.filt=="filter.None"){""}else{exp.filt.opts[[exp.filt]]}}'),

                p_val = 'padj', logFC = 'log2FoldChange',
                label = gene.lab, label.sides = T,


                label.groups='Gene class', label.colors=volc.colors, label.segment.alpha=0.3,
                max.overlaps=20, max.iter=2E6, max.time=25, label.segment.curvature=0) + 
            theme(plot.subtitle=element_text(size=6))

        }) -> p3.phases
        # combine
        p3.combined <- ggarrange(plotlist = p3.phases, nrow=1, align='h', common.legend = T)

        # save
        plot(p3.combined)
    }
    dev.off()
}

[1] "filter.None"


The following `from` values were not present in `x`: TRUE TRUE, FALSE TRUE

The following `from` values were not present in `x`: TRUE TRUE, FALSE TRUE

The following `from` values were not present in `x`: TRUE TRUE, FALSE TRUE

Scale for x is already present.
Adding another scale for x, which will replace the existing scale.
Warning message in geom_text_repel(mapping = repel.aes, size = size.text, segment.alpha = label.segment.alpha, :
“Ignoring unknown aesthetics: nudge_x”
Scale for x is already present.
Adding another scale for x, which will replace the existing scale.
Warning message in geom_text_repel(mapping = repel.aes, size = size.text, segment.alpha = label.segment.alpha, :
“Ignoring unknown aesthetics: nudge_x”
Scale for x is already present.
Adding another scale for x, which will replace the existing scale.
Warning message in geom_text_repel(mapping = repel.aes, size = size.text, segment.alpha = label.segment.alpha, :
“Ignoring unknown aesthetics: nudge_x”
Warning message:
“R

[1] "filter.None"


The following `from` values were not present in `x`: TRUE TRUE, FALSE TRUE



Updading number of genes 30 < 79 ->>> 80


The following `from` values were not present in `x`: TRUE TRUE, FALSE TRUE



Updading number of genes 30 < 54 ->>> 55


The following `from` values were not present in `x`: TRUE TRUE, FALSE TRUE



Updading number of genes 30 < 57 ->>> 60


Scale for x is already present.
Adding another scale for x, which will replace the existing scale.
Warning message in geom_text_repel(mapping = repel.aes, size = size.text, segment.alpha = label.segment.alpha, :
“Ignoring unknown aesthetics: nudge_x”
Scale for x is already present.
Adding another scale for x, which will replace the existing scale.
Warning message in geom_text_repel(mapping = repel.aes, size = size.text, segment.alpha = label.segment.alpha, :
“Ignoring unknown aesthetics: nudge_x”
Scale for x is already present.
Adding another scale for x, which will replace the existing scale.
Warning message in geom_text_repel(mapping = repel.aes, size = size.text, segment.alpha = label.segment.alpha, :
“Ignoring unknown aesthetics: nudge_x”
Warning message:
“Removed 26252 rows containing missing values (`geom_text_repel()`).”
Warning message:
“Removed 26252 rows containing missing values (`geom_text_repel()`).”
Warning message:
“Removed 26292 rows containing missing values (`geom_tex